In [28]:
import geopandas as gpd
import pandas as pd
import concurrent.futures
import multiprocessing
import overpy

In [32]:
# Step 1: Load the city polygon layer
city_layer = gpd.read_file('./Data/city_agglomerations/京津冀/县级.shp', encoding='utf-8')
city_layer['centroid'] = city_layer.centroid
city_layer['coordinates'] = city_layer['centroid'].apply(lambda point: (float(point.x), float(point.y)))

/var/folders/t2/rbnfb0fd63x_nzy8bn0_m4x80000gn/T/ipykernel_26430/3377359710.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  city_layer['centroid'] = city_layer.centroid


In [9]:
api = overpy.Overpass()

In [10]:
def are_cities_connected1(city1, city2):
    query = f"""
    [out:json];
    (area[name="{city1}"]->.a;);
    (area[name="{city2}"]->.b;);
    (
        way(area.a)(area.b)[highway~"motorway|trunk|primary"];
        >;
        (._;>>;);
    );
    out body;
    """
    result = api.query(query)
    return len(result.ways) > 0


In [11]:
are_cities_connected1("东城区","西城区")

True

In [26]:
import requests
import json
def getDistance(start, end):
    start_new = "{},{}".format(round(start[1], 6), round(start[0], 6))
    end_new = "{},{}".format(round(end[1], 6), round(end[0], 6))
    url = "https://api.map.baidu.com/directionlite/v1/driving?origin={}&destination={}&ak={}".format(
        start_new,
        end_new,
        'aIAsQW03aoviegzuvMQ8spuXBG7MfeS8'  # 自动调用，不用修改
    )
    res = requests.get(url)
    json_data = json.loads(res.text)
    
    if json_data["status"] == 0:
        return json_data["result"]["routes"][0]["distance"]
    else:
        print(json_data["message"])
        return -1


In [ ]:
def is_connected(start,end):
    start_new = "{},{}".format(round(start[1], 6), round(start[0], 6))
    end_new = "{},{}".format(round(end[1], 6), round(end[0], 6))
    url = "https://api.map.baidu.com/directionlite/v1/driving?origin={}&destination={}&ak={}".format(
        start_new,
        end_new,
        'aIAsQW03aoviegzuvMQ8spuXBG7MfeS8'  # 自动调用，不用修改
    )
    res = requests.get(url)
    json_data = json.loads(res.text)

    if json_data["status"] == 0:
        road_types = []
        for step in json_data["result"]["routes"][0]["steps"]:
            road_type = step["road_types"]
            road_types.append(road_type)
        road_types = set(city_map.values())
        if road_types.issubset({0, 1, 2, 3}):
            return True
        else:
            return False
    else:
        print(json_data["message"])
        return -1
    

In [27]:
start = 116.20452887099272, 40.21404655628209
end = 116.39868242909261, 39.90918517833325
road_types = connected(start, end)
print("Road Types:", road_types)


Road Types: ['6', '6', '5', '4', '4', '4', '3', '3', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0,1', '1', '1', '1,4', '4', '4', '4', '4', '4', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2,5,2', '2', '2', '2', '2', '2', '2', '2', '2', '4,6', '6', '6', '6']


In [9]:
import osmnx as ox

def are_cities_connected1(city1, city2):
    # Retrieve the street network for the cities
    G = ox.graph_from_place(city1, network_type='drive')  # Network for city1
    H = ox.graph_from_place(city2, network_type='drive')  # Network for city2

    # Check if there is a path between any pair of nodes in the two networks
    connected = any(G.has_edge(u, v) for u, v in H.edges())

    return connected


In [10]:
are_cities_connected1("东城区","西城区")

False

In [11]:
spatial_index = city_layer.sindex

def process_city(args):
    index, city_polygon = args
    origin_city = city_polygon['县']
    origin_code = city_polygon['县代码']
    origin_coords = city_polygon['coordinates']
    neighboring_cities = set()  # Use a set instead of a list

    # Find neighboring cities based on spatial relationship using the spatial index
    neighboring_indices = list(spatial_index.intersection(city_polygon.geometry.bounds))
    for other_index in neighboring_indices:
        other_city_polygon = city_layer.iloc[other_index]
        if index != other_index and city_polygon.geometry.touches(other_city_polygon.geometry):
            neighboring_city = other_city_polygon['县']
            neighboring_code = other_city_polygon['县代码']
            if are_cities_connected1(origin_city, neighboring_city):
                neighboring_cities.add((neighboring_city, neighboring_code))
            else:
                continue

    return [(origin_city, origin_code, origin_coords, neighboring_city, neighboring_code)
            for neighboring_city, neighboring_code in neighboring_cities]

In [6]:
from city_processing import process_city
results = []
# Process cities in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    args = [(index, city_polygon, city_layer) for index, city_polygon in city_layer.iterrows()]
    results = list(executor.map(process_city,args))

禅城区 and 顺德区 are connected
禅城区 and 南海区 are connected
顺德区 and 蓬江区 are connected
南海区 and 鹤山市 are connected
三水区 and 高明区 are connected
东莞市 and 龙华区 are connected
高明区 and 鹤山市 are connected
高明区 and 南海区 are connected三水区 and 南海区 are connected

东莞市 and 宝安区 are connected
高明区 and 高要区 are connected
三水区 and 高要区 are connected
顺德区 and 中山市 are connected
顺德区 and 南沙区 are connected
东莞市 and 南沙区 are connected
从化区 and 白云区 are connected
白云区 and 南海区 are connected
东莞市 and 光明区 are connected
顺德区 and 禅城区 are connected
海珠区 and 荔湾区 are connected
海珠区 and 越秀区 are connected
南海区 and 高明区 are connected
番禺区 and 南沙区 are connected
番禺区 and 东莞市 are connected
三水区 and 花都区 are connected
黄埔区 and 东莞市 are connected
高明区 and 三水区 are connected
顺德区 and 南海区 are connected
东莞市 and 龙岗区 are connected
白云区 and 荔湾区 are connected
从化区 and 龙门县 are connected
顺德区 and 番禺区 are connected
荔湾区 and 南海区 are connected
白云区 and 越秀区 are connected
三水区 and 四会市 are connected
番禺区 and 顺德区 are connected
越秀区 and 荔湾区 are connected
南沙区 and 中山市 are connected
黄埔区 and 白云区 

OverpassGatewayTimeout: Server load too high

In [24]:
# Convert the results list to a DataFrame
table = pd.DataFrame(results, columns=['Origin', 'OrigCode', 'OrigCoord', 'Dest', 'DestCode'])
#table = table.explode('Dest').rename(columns={'Dest': 'DestCode'}).reset_index(drop=True)

ValueError: 5 columns passed, passed data had 0 columns

In [32]:
table

,Origin,OrigCode,OrigCoord,Dest,DestCode
0,昌平区,110114,"(116.20452887099272, 40.21404655628209)",[],[]
1,朝阳区,110105,"(116.50935364028471, 39.95234024823175)",[],[]
2,大兴区,110115,"(116.41297001077022, 39.64648069987429)",[],[]
3,东城区,110101,"(116.41050197143848, 39.91152786958014)",[],[]
4,房山区,110111,"(115.84779135463242, 39.71787758197887)",[],[]
...,...,...,...,...,...
203,静海区,120118,"(116.9603018394033, 38.85473052134871)",[],[]
204,南开区,120104,"(117.14867236468932, 39.108252038866475)",[],[]
205,宁河区,120117,"(117.6506147282753, 39.376568207369644)",[],[]
206,武清区,120114,"(117.02205842508047, 39.44980461653322)",[],[]


In [15]:
table.to_csv("./Data/city_agglomerations/京津冀/BTJ_network.csv")

Flows

In [30]:
import requests
import json
import time
import xlsxwriter

In [33]:
city_layer

,省,省代码,市,市代码,县,县代码,说明,年度,县类型,市类型,省类型,数量,geometry,centroid,coordinates
0,北京市,110000,北京市,110000,昌平区,110114,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((116.49992 40.26013, 116.49859 40.259...",POINT (116.20453 40.21405),"(116.20452887099272, 40.21404655628209)"
1,北京市,110000,北京市,110000,朝阳区,110105,None,0,市辖区,直辖市,直辖市,1,"MULTIPOLYGON (((116.47730 40.08196, 116.47685 ...",POINT (116.50935 39.95234),"(116.50935364028471, 39.95234024823175)"
2,北京市,110000,北京市,110000,大兴区,110115,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((116.52906 39.82371, 116.52906 39.823...",POINT (116.41297 39.64648),"(116.41297001077022, 39.64648069987429)"
3,北京市,110000,北京市,110000,东城区,110101,北京市人民政府驻地由东城区正义路2号迁至通州区运河东大街57号。,2018,市辖区,直辖市,直辖市,1,"POLYGON ((116.43745 39.87148, 116.43740 39.871...",POINT (116.41050 39.91153),"(116.41050197143848, 39.91152786958014)"
4,北京市,110000,北京市,110000,房山区,110111,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((116.04836 39.84460, 116.05316 39.842...",POINT (115.84779 39.71788),"(115.84779135463242, 39.71787758197887)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,天津市,120000,天津市,120000,静海区,120118,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((116.87565 39.07094, 116.88614 39.067...",POINT (116.96030 38.85473),"(116.9603018394033, 38.85473052134871)"
204,天津市,120000,天津市,120000,南开区,120104,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((117.18813 39.14594, 117.18842 39.145...",POINT (117.14867 39.10825),"(117.14867236468932, 39.108252038866475)"
205,天津市,120000,天津市,120000,宁河区,120117,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((117.92790 39.57310, 117.92505 39.567...",POINT (117.65061 39.37657),"(117.6506147282753, 39.376568207369644)"
206,天津市,120000,天津市,120000,武清区,120114,None,0,市辖区,直辖市,直辖市,1,"POLYGON ((117.14141 39.62126, 117.14203 39.619...",POINT (117.02206 39.44980),"(117.02205842508047, 39.44980461653322)"


In [34]:
CitiesCode = {}
for index, row in city_layer.iterrows():
    city_name = row["县"]  # Replace "city_name_column" with the actual column name in the shapefile
    city_code = row["县代码"]  # Replace "city_code_column" with the actual column name in the shapefile
    CitiesCode[city_name] = city_code


In [35]:
CitiesCode

{'昌平区': 110114,
 '朝阳区': 110105,
 '大兴区': 110115,
 '东城区': 110101,
 '房山区': 110111,
 '丰台区': 110106,
 '海淀区': 110108,
 '怀柔区': 110116,
 '门头沟区': 110109,
 '密云区': 110118,
 '平谷区': 110117,
 '石景山区': 110107,
 '顺义区': 110113,
 '通州区': 110112,
 '西城区': 110102,
 '延庆区': 110119,
 '安国市': 130683,
 '安新县': 130632,
 '博野县': 130637,
 '定兴县': 130626,
 '定州市': 130682,
 '阜平县': 130624,
 '高碑店市': 130684,
 '高阳县': 130628,
 '竞秀区': 130602,
 '涞水县': 130623,
 '涞源县': 130630,
 '蠡县': 130635,
 '莲池区': 130606,
 '满城区': 130607,
 '清苑区': 130608,
 '曲阳县': 130634,
 '容城县': 130629,
 '顺平县': 130636,
 '唐县': 130627,
 '望都县': 130631,
 '雄县': 130638,
 '徐水区': 130609,
 '易县': 130633,
 '涿州市': 130681,
 '沧县': 130921,
 '东光县': 130923,
 '海兴县': 130924,
 '河间市': 130984,
 '黄骅市': 130983,
 '孟村回族自治县': 130930,
 '南皮县': 130927,
 '泊头市': 130981,
 '青县': 130922,
 '任丘市': 130982,
 '肃宁县': 130926,
 '吴桥县': 130928,
 '献县': 130929,
 '新华区': 130105,
 '盐山县': 130925,
 '运河区': 130903,
 '承德县': 130821,
 '丰宁满族自治县': 130826,
 '宽城满族自治县': 130827,
 '隆化县': 130825,
 '滦平县': 130824,
 '平泉市': 130881,


In [38]:
def migration_index(FileTittle, classname, direction, CodeDict):
    if direction == 'in':
        nameofdire = '迁入'
    elif direction == 'out':
        nameofdire = '迁出'

    workbook = xlsxwriter.Workbook(f'{FileTittle} {nameofdire}规模指数.xlsx')
    worksheet = workbook.add_worksheet('Sheet')

    if direction == 'in':
        nameofdire = '迁入'
    elif direction == 'out':
        nameofdire = '迁出'

    CitiesOrder = {}
    worksheet.write(0, 0, '城市代码')
    worksheet.write(0, 1, '城市')
    times = 1

    for key, value in CodeDict.items():
        worksheet.write(times, 0, str(value))
        worksheet.write(times, 1, str(key))
        CitiesOrder[str(key)] = times
        times += 1

    for key, value in CodeDict.items():
        url = f'http://huiyan.baidu.com/migration/historycurve.jsonp?dt={classname}&id={value}&type=move_{direction}'
        print(f'{key}:{url}')
        response = requests.get(url, timeout=2)
        time.sleep(3)
        r = response.text[3:-1]
        
        try:
            data_dict = json.loads(r)

            if data_dict['errmsg'] == 'SUCCESS':
                data_list = data_dict['data']['list']
                counter_date = 2
                datelist = []

                for date, index in data_list.items():
                    datelist.append(date)

                datelist.sort()

                for date in datelist:
                    index = data_list[date]
                    worksheet.write(0, counter_date, float(date))
                    worksheet.write(CitiesOrder[str(key)], counter_date, float(index))
                    counter_date += 1

            else:
                print('Error')

        except json.JSONDecodeError:
            print('Failed to decode JSON response:', r)

    workbook.close()

In [39]:
if __name__ == "__main__":
    for key, value in CitiesCode.items():
        migration_index(key, 'city', 'in', CitiesCode)
        migration_index(key, 'city', 'out', CitiesCode)

    print('All done')

昌平区:http://huiyan.baidu.com/migration/historycurve.jsonp?dt=city&id=110114&type=move_in
Failed to decode JSON response: ({"errno":0,"errmsg":"SUCCESS","data":{"list":[]}}
朝阳区:http://huiyan.baidu.com/migration/historycurve.jsonp?dt=city&id=110105&type=move_in
Failed to decode JSON response: ({"errno":0,"errmsg":"SUCCESS","data":{"list":[]}}
大兴区:http://huiyan.baidu.com/migration/historycurve.jsonp?dt=city&id=110115&type=move_in


KeyboardInterrupt: 